## DANNY'S DINER SQL CHALLENGE
This is a sql project created by [Danny Ma]("https://www.linkedin.com/in/datawithdanny/") to help individuals emerging into the data scene. From my view, it helps establish the foundational knowledge of sql while testing and developing logical problem skills.
I have chosen to do it in Python because it highlights how you can create a database on your system to allow for continuous use.

To get access to this and other of Danny's projects - [8 Week SQL Challenge]("https://8weeksqlchallenge.com/").

Before getting started, I would also recommend installing "ipython-sql". This allows you use the 'jupyter magic' function to interact with your relational database.

### Importing Libraries

In [1]:
import sqlite3 as sql
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

### Creating Database, Table & Inserting Values into Table

In [2]:
conn = sql.connect('dannys_diner.db')
c = conn.cursor()

def create_table():
    c.execute('CREATE TABLE IF NOT EXISTS sales (customer_id TEXT,order_date TEXT,product_id INT)')
    c.execute('CREATE TABLE IF NOT EXISTS menu (product_id INT,product_name TEXT,price REAL)')
    c.execute('CREATE TABLE IF NOT EXISTS members (customer_id TEXT,join_date TEXT)')
              
def data_entry():
    c.execute("INSERT INTO sales VALUES ('A', '2021-01-01', 1)")
    c.execute("INSERT INTO sales VALUES ('A', '2021-01-01', 2)")
    c.execute("INSERT INTO sales VALUES ('A', '2021-01-07', 2)")
    c.execute("INSERT INTO sales VALUES ('A', '2021-01-10', 3)")
    c.execute("INSERT INTO sales VALUES ('A', '2021-01-11', 3)")
    c.execute("INSERT INTO sales VALUES ('A', '2021-01-11', 3)")
    c.execute("INSERT INTO sales VALUES ('B', '2021-01-01', 2)")
    c.execute("INSERT INTO sales VALUES ('B', '2021-01-02', 2)")
    c.execute("INSERT INTO sales VALUES ('B', '2021-01-04', 1)")
    c.execute("INSERT INTO sales VALUES ('B', '2021-01-11', 1)")
    c.execute("INSERT INTO sales VALUES ('B', '2021-01-16', 3)")
    c.execute("INSERT INTO sales VALUES ('B', '2021-02-01', 3)")
    c.execute("INSERT INTO sales VALUES ('C', '2021-01-01', 3)")
    c.execute("INSERT INTO sales VALUES ('C', '2021-01-01', 3)")
    c.execute("INSERT INTO sales VALUES ('C', '2021-01-07', 3)")
    
    c.execute("INSERT INTO menu VALUES (1, 'sushi', 10)")
    c.execute("INSERT INTO menu VALUES (2, 'curry', 15)")
    c.execute("INSERT INTO menu VALUES (3, 'ramen', 12)")
              
    c.execute("INSERT INTO members VALUES ('A', '2021-01-07')")
    c.execute("INSERT INTO members VALUES ('B', '2021-01-09')")
    
    conn.commit()
    conn.close()
    
create_table()
data_entry()

In [3]:
%%capture
%load_ext sql
%sql sqlite:///dannys_diner.db

In [4]:
%%sql
SELECT 
* 
FROM sales;

 * sqlite:///dannys_diner.db
Done.


customer_id,order_date,product_id
A,2021-01-01,1
A,2021-01-01,2
A,2021-01-07,2
A,2021-01-10,3
A,2021-01-11,3
A,2021-01-11,3
B,2021-01-01,2
B,2021-01-02,2
B,2021-01-04,1
B,2021-01-11,1


In [5]:
%%sql
SELECT 
* 
FROM menu;

 * sqlite:///dannys_diner.db
Done.


product_id,product_name,price
1,sushi,10.0
2,curry,15.0
3,ramen,12.0


In [6]:
%%sql
SELECT 
* 
FROM members;

 * sqlite:///dannys_diner.db
Done.


customer_id,join_date
A,2021-01-07
B,2021-01-09


### Case Study Questions

#### Question 1 - What is the total amount each customer spent at the restaurant?

In [7]:
%%sql
select s.customer_id, sum(price) total_spent
from sales s
left join menu m
on s.product_id = m.product_id
group by s.customer_id;

 * sqlite:///dannys_diner.db
Done.


customer_id,total_spent
A,76.0
B,74.0
C,36.0


- Customer `A` spent a total of `$76.0`
- Customer `B` spent a total of `$74.0`
- Customer `C` spent a total of `$36.0`

#### Question 2 - How many days has each customer visited the restaurant?

In [8]:
%%sql
select s.customer_id, count(distinct s.order_date) days_visited
from sales s
group by s.customer_id;

 * sqlite:///dannys_diner.db
Done.


customer_id,days_visited
A,4
B,6
C,2


- Customer `A` has visited the restaurant `4` times
- Customer `B` has visited the restaurant `6` times
- Customer `C` has visited the restaurant `2` times.

#### Question 3 - What was the first item from the menu purchased by each customer?

In [9]:
%%sql
with first_order as (
select s.customer_id, s.order_date, m.product_name
, row_number() over (partition by s.customer_id order by s.order_date) Rank
from sales s
left join menu m
on s.product_id = m.product_id
) 
        
select f.customer_id, f.order_date,f.product_name
from first_order f
where f.rank = 1;

 * sqlite:///dannys_diner.db
Done.


customer_id,order_date,product_name
A,2021-01-01,sushi
B,2021-01-01,curry
C,2021-01-01,ramen


- `Sushi` was the first item purchased by Customer `A`
- `Curry` was the first item purchased by Customer `B`
- `Ramen` was the first item purchased by Customer `C`.

#### Question 4 - What is the most purchased item on the menu and how many times was it purchased by all customers?

In [10]:
%%sql
select m.product_name, count(m.product_name) count
from sales s
left join menu m
on s.product_id = m.product_id
group by m.product_name
order by count desc;

 * sqlite:///dannys_diner.db
Done.


product_name,count
ramen,8
curry,4
sushi,3


- `Ramen` is the most purchased item on the menu and it was purchased `8` times by all customers.

#### Question 5 - Which item was the most popular for each customer?

In [11]:
%%sql
select mp.customer_id, mp.product_name, max(mp.CountofOrders) most_popular
from(
select s.customer_id, m.product_name
, count(m.product_name) over (partition by s.customer_id,m.product_name ) CountofOrders
from sales s
left join menu m
on s.product_id = m.product_id
) mp
group by mp.customer_id;

 * sqlite:///dannys_diner.db
Done.


customer_id,product_name,most_popular
A,ramen,3
B,curry,2
C,ramen,3


- `Popularity` is defined by the most purchased item
- `Ramen` is customer `A's` most popular item
- Customer `B` on the other hand had the same count for all their purchased items. Thus, based on the first item they bought (`which is subjective`), `Curry` is their most popular item
- `Ramen` is also customer `C's` most popular item.

#### Question 6 - Which item was purchased first by the customer after they became a member?

In [12]:
%%sql
with mem as (
select s.customer_id, m.product_name, s.order_date, m1.join_date
, row_number() over (partition by s.customer_id order by s.order_date) Rank
from sales s
left join menu m
on s.product_id = m.product_id
inner join members m1
on s.customer_id = m1.customer_id
where s.order_date >= m1.join_date 
)

select m.customer_id, m.product_name, m.order_date, m.join_date
from mem m
where Rank = 1;

 * sqlite:///dannys_diner.db
Done.


customer_id,product_name,order_date,join_date
A,curry,2021-01-07,2021-01-07
B,sushi,2021-01-11,2021-01-09


- After they became a member, customer `A's` first order was `curry`
- Customer `B's` first order after membership was `sushi`
- Customer `C` on the other hand never became a member.

#### Question 7 - Which item was purchased just before the customer became a member?

In [13]:
%%sql
with mem as (
select s.customer_id, m.product_name, s.order_date, m1.join_date
, row_number() over (partition by s.customer_id order by s.order_date) Rank
from sales s
left join menu m
on s.product_id = m.product_id
inner join members m1
on s.customer_id = m1.customer_id
where s.order_date < m1.join_date 
)

select m.customer_id, m.product_name, m.order_date, m.join_date
from mem m
group by m.customer_id
having m.Rank = max(m.Rank);

 * sqlite:///dannys_diner.db
Done.


customer_id,product_name,order_date,join_date
A,curry,2021-01-01,2021-01-07
B,sushi,2021-01-04,2021-01-09


- Customer `A` purchased `curry` just before they became a member
- Customer `B` purchased `sushi` just before they became a member.

#### Question 8 - What is the total items and amount spent for each member before they became a member?

In [14]:
%%sql
with b4_mem as (
select s.customer_id, m.product_name, m.price, s.order_date, m1.join_date
, row_number() over (partition by s.customer_id order by s.order_date) Rank
from sales s
left join menu m
on s.product_id = m.product_id
inner join members m1
on s.customer_id = m1.customer_id
where s.order_date < m1.join_date 
)

select m.customer_id, count(m.product_name) total_items, sum(m.price) total_spent
from b4_mem m
group by m.customer_id;

 * sqlite:///dannys_diner.db
Done.


customer_id,total_items,total_spent
A,2,25.0
B,3,40.0


- Prior to their membership, customer `A` purchased a total of `2` items and spent `$25` in total
- Customer `B` purchased a total of `3` items and spent `$40` GBP in total prior to their membership
- Customer `C` on the other hand never became a member and so they are excluded from the result.

#### Question 9 - If each $1 spent equates to 10 points and sushi has a 2x points multiplier - how many points would each customer have?

In [15]:
%%sql
select s.customer_id
, cast(sum(case m.product_name when 'sushi' then m.price * 2 else m.price * 1 end) as int) points
from sales s
left join menu m
on s.product_id = m.product_id
group by s.customer_id;

 * sqlite:///dannys_diner.db
Done.


customer_id,points
A,86
B,94
C,36


- Customer `A` would have `86` points
- Customer `B` would have `94` points 
- Customer `C` would have `36` points.

#### Question 10 - In the first week after a customer joins the program (including their join date) they earn 2x points on all items, not just sushi - how many points do customer A and B have at the end of January?

In [16]:
%%sql
select s.customer_id
, cast(sum(case when s.order_date between date(m1.join_date) and date(m1.join_date,'+6 day') 
then m.price * 2 else m.price * 1 end) as int) points
from sales s
left join menu m
on s.product_id = m.product_id
inner join members m1
on s.customer_id = m1.customer_id
where s.order_date between m1.join_date and '2021-01-31'
group by s.customer_id;

 * sqlite:///dannys_diner.db
Done.


customer_id,points
A,102
B,32


- Customer `A` has `102` points
- Customer `B` has `32` points.

#### Bonus Question - Join All Things

##### Create table holding all essential data to avoid joining tables

In [17]:
%%sql
CREATE VIEW full_danny
AS

select s.customer_id, s.order_date, m.product_name, m.price
,case when s.order_date >= m1.join_date then 'Y' else 'N' end member
from sales s
left join menu m
on s.product_id = m.product_id
left join members m1
on s.customer_id = m1.customer_id

 * sqlite:///dannys_diner.db
Done.


[]

In [18]:
%%sql
select * 
from 
full_danny

 * sqlite:///dannys_diner.db
Done.


customer_id,order_date,product_name,price,member
A,2021-01-01,sushi,10.0,N
A,2021-01-01,curry,15.0,N
A,2021-01-07,curry,15.0,Y
A,2021-01-10,ramen,12.0,Y
A,2021-01-11,ramen,12.0,Y
A,2021-01-11,ramen,12.0,Y
B,2021-01-01,curry,15.0,N
B,2021-01-02,curry,15.0,N
B,2021-01-04,sushi,10.0,N
B,2021-01-11,sushi,10.0,Y


In [19]:
%%sql
select *  
, case when member = 'N' 
then 'null' else
rank() over (partition by customer_id,member order by order_date asc) end ranking
from 
full_danny

 * sqlite:///dannys_diner.db
Done.


customer_id,order_date,product_name,price,member,ranking
A,2021-01-01,sushi,10.0,N,null
A,2021-01-01,curry,15.0,N,null
A,2021-01-07,curry,15.0,Y,1
A,2021-01-10,ramen,12.0,Y,2
A,2021-01-11,ramen,12.0,Y,3
A,2021-01-11,ramen,12.0,Y,3
B,2021-01-01,curry,15.0,N,null
B,2021-01-02,curry,15.0,N,null
B,2021-01-04,sushi,10.0,N,null
B,2021-01-11,sushi,10.0,Y,1
